<a href="https://colab.research.google.com/github/yogesh7122002/BackEnd-Task/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# @title
!pip install langchain langchain-openai langchain-community chromadb

In [8]:
from google.colab import userdata

In [9]:
import  os
OPENAI_API_KEY = userdata.get('OPENAI_API')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [10]:
import re

def clean_data( raw_text: str):
    try:
        text = raw_text.encode('latin1').decode('utf-8')
    except Exception:
        text = raw_text
    #Remove unwanted Unicode symbols like \uf058
    text = re.sub(r'[\uE000-\uF8FF]', '', text)

    # Remove unicode escapes like \u00a0
    # text = re.sub(r'\\u[0-9a-fA-F]{4}', ' ', text)

    # Remove multiple spaces/newlines
    text = re.sub(r'\s+', ' ', text)

    # Strip unwanted control characters

    text = re.sub(r'Home|Products|Pricing|About|Contact|','',text,flags=re.IGNORECASE)
    text = text[:-336]
    text = text.strip()
    return text


In [11]:
from langchain_community.document_loaders import WebBaseLoader
page_urls = ["https://datadios.ai/datadiff/","https://datadios.ai/data-quality-dashboard/"] #,
loader = WebBaseLoader(web_paths=page_urls)
# docs = []
# async for doc in loader.alazy_load():
#   docs.append(doc)
docs = loader.load()
# print(docs)
# data = []
# for doc in docs:
#   data.append(clean_data(doc.page_content))
  # print(doc.page_content)
# data = clean_data(docs)

In [12]:
# data

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap = 200
)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'https://datadios.ai/datadiff/', 'title': 'datadiff | DataDios', 'language': 'en-US'}, page_content='datadiff | DataDios\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHomeProducts\n\nMeta VisionAI-StudioData ExplorerWorkload Analyzer\r\nData Quality DashboardUniversal Semantic SearchSmartDiffMetadata Synchronization\nPricingAboutBlogsContact \n\n\nSelect Page\n\n\n  \n \n \n\n\n\n\n\n\n\n\n\n\nSmartDiff\n\n\n\n\n\nDataDios SmartDiff ensures an easy, efficient, and secure way to validate migrated data across private and public cloud platforms. DataDios SmartDiff is built on Root Cause Analysis, Clustering and Data Transformation architecture enables automated Data Validation Post Migrations, Cause analysis and revealing the patterns\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSingle Dashboard\n\n\n\n\n\nWe provide a single dashboard to perform data diff, metadata diff and products diff irrespective of:'),
 Document(meta

In [14]:
#create the vector store and store  the data into it
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
vector_store

In [15]:
vector_store._collection.count()
vector_store._collection.get()

{'ids': ['460afab1-bb99-46ab-84d5-ef5de531a730',
  'bc3b26ee-9618-462d-814b-478341f08dec',
  '1e483893-d706-400e-bf21-fa451876ef4d',
  '55cd72e4-6959-4ba7-9fa2-d2ba001b62a5',
  '67cb5308-d0c9-447c-8a44-d852647396b6',
  'e27c3447-e290-4e78-9f21-8cd0bf9544e0',
  '21c4f2bd-b605-401d-83c8-98ec60c8f591',
  '4abca3ec-579c-45f2-b4de-1770f5695a71',
  'a65b0588-ba7f-41e6-ba6d-6e86cb712a42',
  '5ad7a79c-91a3-4c63-adff-96da8ff52f51',
  '33f9a5d1-241e-492b-9554-d14d291fcc6a',
  '74b9f133-f32d-48c3-99ac-cda63c4f45e0',
  '5191515f-6360-4166-8442-1c2f5187806f',
  '9381d81a-c2e4-48b9-9fed-1280cc737ba0'],
 'embeddings': None,
 'documents': ['datadiff | DataDios\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHomeProducts\n\nMeta VisionAI-StudioData ExplorerWorkload Analyzer\r\nData Quality DashboardUniversal Semantic SearchSmartDiffMetadata Synchronization\nPricingAboutBlogsContact \n\n\nSelect Page\n\n\n  \n \n \n\n\n\n\n\n\n\n\n\n\nSmartDiff\n\n\n\n\n\

In [16]:
# Add retriver to retrive the data from the storage
retriver = vector_store.as_retriever()


In [17]:
# there are prompts on hub so we  are using it  from hub
#i can use the my local also

from langchain import hub
prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [18]:
#create the chat Model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() #use the default name

In [19]:
#whenever we want to pass anything here as it is then we use the runnable pass through from langchain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [20]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [21]:
rag_chain = ({"context":retriver | format_docs,"question":RunnablePassthrough()}
              | prompt
              | llm
              | StrOutputParser()
              )

In [22]:
rag_chain.invoke("what is smartdiff and how it's helps for data validation")

'SmartDiff is a tool that provides a balanced, multi-cloud strategy to validate migrated data across private and public cloud platforms. It is built on Root Cause Analysis, Clustering, and Data Transformation architecture for automated data validation post-migrations. SmartDiff helps in cost-efficient validation, easy-to-use graphical interface, optimization, and meeting project deadlines.'